* [](http://)VGG16  com Regularization + Data Augmentation + Dropout 0.5

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(2019)

%matplotlib inline
import matplotlib.pyplot as plt

import keras
from keras import regularizers
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten


import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
from keras.preprocessing.image import ImageDataGenerator

Reading csv's to get the names of the images and their breeds.

In [ ]:
df_train = pd.read_csv('../input/dog-breed-identification/labels.csv') # arquivo com os nomes das imagens e as suas raças para o treino
df_test = pd.read_csv('../input/dog-breed-identification/sample_submission.csv') # arquivo com os nomes das imagens de teste

In [ ]:
df_train.head()

In [ ]:
df_test.head()

Races must be using * one-hot encode *

In [ ]:
targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [ ]:
one_hot_labels = np.asarray(one_hot)

The following will read the training and test images.

In [ ]:
im_size = 197

In [ ]:
x_train = []
y_train = []
x_test = []

In [ ]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = cv2.imread('../input/dog-breed-identification/train/{}.jpg'.format(f))
    x_train.append(cv2.resize(img, (im_size, im_size)))
    label = one_hot_labels[i]
    y_train.append(label)
    i += 1

In [ ]:
# delete df_train to decrease memory usage
del df_train

In [ ]:
for f in tqdm(df_test['id'].values):
    img = cv2.imread('../input/dog-breed-identification/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))

In [ ]:
num_class = 120

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train,shuffle=True,  test_size=0.2, random_state=1)

In [ ]:
datagen = ImageDataGenerator(width_shift_range=0.2,
                            height_shift_range=0.2,
                            zoom_range=0.2,
                            rotation_range=30,
                            vertical_flip=False,
                            horizontal_flip=True)

datagen.fit(X_train)

In [ ]:
train_generator = datagen.flow(np.array(X_train), np.array(Y_train), 
                               batch_size=32) 

In [ ]:
def create_my_model(use_regularizer, optimizer):
    base_model = VGG16(weights="../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5",include_top=False, input_shape=(im_size, im_size, 3))
    dropout = base_model.output
    dropout = Dropout(0.5)(dropout)
    model_with_dropout = Model(inputs=base_model.input, outputs=dropout)
        
    x = base_model.output
    x = Flatten()(x)
    predictions = Dense(num_class, activation='softmax', kernel_regularizer=regularizers.l2(0.0015), activity_regularizer=regularizers.l1(0.0015))(x)
    
    my_model = Model(inputs=base_model.input, outputs=predictions)
    
#     for layer in my_model.layers:
#         layer.treinable = False
    
    my_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return my_model


In [ ]:
def gerar_grafico(historia, titulo):
    plt.plot(historia.history['acc'])
    plt.plot(historia.history['val_acc'])
    plt.title('Accuracy ' + titulo)
    plt.ylabel('Accuracy')
    plt.xlabel('Times')
    plt.legend(['training', 'validation'], loc='upper left')
    plt.show()
    plt.plot(historia.history['loss'])
    plt.plot(historia.history['val_loss'])
    plt.title('Loss ' + titulo)
    plt.ylabel('Loss')
    plt.xlabel('Times')
    plt.legend(['training', 'validation'], loc='upper left')
    plt.show()

In [ ]:
model_rmsprop_com_regularizador = create_my_model(use_regularizer=True, optimizer='rmsprop')
model_sgd_com_regularizador = create_my_model(use_regularizer=True, optimizer='sgd')

In [ ]:
history_rmsprop_com_regularizador = model_rmsprop_com_regularizador.fit_generator(
    train_generator,
    epochs=10, steps_per_epoch=len(X_train) / 18, #len(X_train) / 18,
    validation_data=(np.array(X_train), np.array(Y_train)), validation_steps=len(X_valid) / 18)

preds = model_rmsprop_com_regularizador.predict(np.array(x_test), verbose=1)

gerar_grafico(history_rmsprop_com_regularizador, 
              "VGG16 with RMSprop")

sub = pd.DataFrame(preds)
col_names = one_hot.columns.values
sub.columns = col_names
sub.insert(0, 'id', df_test['id'])
sub.head(5)

sub.to_csv("predictions_vgg16_with_RMSProp.csv")

model_rmsprop_com_regularizador.save('vgg16_with_RMSProp.h5')


In [ ]:
history_sgd_com_regularizador = model_sgd_com_regularizador.fit_generator(
    train_generator,
    epochs=10, steps_per_epoch=len(X_train) / 18, #len(X_train) / 18,
    validation_data=(np.array(X_train), np.array(Y_train)), validation_steps=len(X_valid) / 18)

preds = model_sgd_com_regularizador.predict(np.array(x_test), verbose=1)

gerar_grafico(history_sgd_com_regularizador, 
              "VGG16 com SGD")

sub = pd.DataFrame(preds)
col_names = one_hot.columns.values
sub.columns = col_names
sub.insert(0, 'id', df_test['id'])
sub.head(5)

sub.to_csv("output_sgd_v2_com_data_augmentation_e_sem_regularizador.csv")

model_sgd_com_regularizador.save('sgd_v2_com_data_augmentation_e_sem_regularizador.h5')

In [ ]:
from sklearn.metrics import f1_score

f1_score(Y_train, Y_valid, average='macro') 

In [ ]:
f1_score(Y_train, Y_valid, average='micro') 

In [ ]:
f1_score(Y_train, Y_valid, average='weighted') 

In [ ]:
f1_score(Y_train, Y_valid, average=None) 

In [ ]:
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,
                             f1_score)
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.model_selection import train_test_split


# Create dataset of classification task with many redundant and few
# informative features
#X, y = datasets.make_classification(n_samples=100000, n_features=20,
                                    #n_informative=2, n_redundant=10,
                                    #random_state=42)
#X_train, X_valid, Y_train, Y_valid # => 
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.99,
                                                    #random_state=42)


def plot_calibration_curve(est, name, fig_index):
    """Plot calibration curve for est w/o and with calibration. """
    # Calibrated with isotonic calibration
    isotonic = CalibratedClassifierCV(est, cv=2, method='isotonic')

    # Calibrated with sigmoid calibration
    sigmoid = CalibratedClassifierCV(est, cv=2, method='sigmoid')

    # Logistic regression with no calibration as baseline
    lr = LogisticRegression(C=1.)

    fig = plt.figure(fig_index, figsize=(10, 10))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax2 = plt.subplot2grid((3, 1), (2, 0))

    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    for clf, name in [(lr, 'Logistic'),
                      (est, name),
                      (isotonic, name + ' + Isotonic'),
                      (sigmoid, name + ' + Sigmoid')]:
        clf.fit(X_train, Y_train)
        y_pred = clf.predict(X_valid)
        if hasattr(clf, "predict_proba"):
            prob_pos = clf.predict_proba(X_valid)[:, 1]
        else:  # use decision function
            prob_pos = clf.decision_function(X_valid)
            prob_pos = \
                (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())

        clf_score = brier_score_loss(Y_valid, prob_pos, pos_label=y.max())
        print("%s:" % name)
        print("\tBrier: %1.3f" % (clf_score))
        print("\tPrecision: %1.3f" % precision_score(Y_valid, y_pred))
        print("\tRecall: %1.3f" % recall_score(Y_valid, y_pred))
        print("\tF1: %1.3f\n" % f1_score(Y_valid, y_pred))

        fraction_of_positives, mean_predicted_value = \
            calibration_curve(Y_valid, prob_pos, n_bins=10)

        ax1.plot(mean_predicted_value, fraction_of_positives, "s-",
                 label="%s (%1.3f)" % (name, clf_score))

        ax2.hist(prob_pos, range=(0, 1), bins=10, label=name,
                 histtype="step", lw=2)

    ax1.set_ylabel("Fraction of positives")
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc="lower right")
    ax1.set_title('Calibration plots  (reliability curve)')

    ax2.set_xlabel("Mean predicted value")
    ax2.set_ylabel("Count")
    ax2.legend(loc="upper center", ncol=2)

    plt.tight_layout()

# Plot calibration curve for Gaussian Naive Bayes
plot_calibration_curve(GaussianNB(), "Naive Bayes", 1)

# Plot calibration curve for Linear SVC
plot_calibration_curve(LinearSVC(max_iter=10000), "SVC", 2)

plt.show()